In [38]:
# now the decomposition prompt is done

In [39]:
with open("examples/rap_gsm8k_l2m/prompts/decomposition.txt") as f:
    data = f.read()

In [31]:
cases = [d.split("\n") for d in data.split("\n\n")]

In [32]:
processed_data = []

In [33]:
st = ""
for c in cases:
    st += c[0][3:] + "\n\n"
    
    sub = c[1].split("need to know: ")[-1].split(",")
    final = c[1].split("To answer the question \"")[-1].split("\", we need to")[0]
    for s in sub + [" " + final + " "]:
        s = s[1:-1]
        if s.startswith("\""):
            s = s[1:]
        if s.endswith("\""):
            s = s[:-1]
        st += "Q: " + s + "\nA: The answer is ."+"\n\n"

In [34]:
with open("examples/rap_gsm8k_l2m/prompts/solving.txt", "w") as f:
    f.write(st)

In [ ]:
# after labeling `solving.txt`
# build the final file

In [42]:
with open("examples/rap_gsm8k_l2m/prompts/solving.txt") as f:
    solving_data = f.read()

In [37]:
import json

original = json.load(open("examples/rap_gsm8k_l2m/prompts/l2m.json"))

In [52]:
revised = original
decomposition_pool = [
    d.strip() + "\n\n" for d in data.split("\n\n")
]

In [55]:
solving_pool = []

cache = ""
for d in solving_data.split("\n\n"):
    d = d.strip()
    if d == "":
        continue
    if not d.startswith("Q:"):
        if cache != "":
            solving_pool.append(cache)
        cache = d.strip() + "\n\n"
    else:
        cache += d.strip() + "\n\n"
solving_pool.append(cache)

In [56]:
solving_pool

['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\n\nQ: How many clips did Natalia sell in May?\nA: Natalia sold 48 clips in April and half as many clips in May, so she sold 48 / 2 = 24 clips in May. The answer is 24.\n\nQ: How many clips did Natalia sell altogether in April and May?\nA: Natalia sold 48 clips in April and 24 clips in May, so altogether she sold 48 + 24 = 72 clips. The answer is 72.\n\n',
 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?\n\nQ: How much does Weng earn per minute?\nA: Since Weng earns $12 an hour for babysitting, she earns $12 / 60 = $0.2 per minute. The answer is 0.2.\n\nQ: How much did she earn?\nA: Working 50 minutes, she earned $0.2 x 50 = $10. The answer is 10.\n\n',
 "Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents

In [41]:
decomposition_pool

['Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\nA: To answer the question "How many clips did Natalia sell altogether in April and May?", we need to know: "How many clips did Natalia sell in May?".\n\n',
 'Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?\nA: To answer the question "How much did she earn?", we need to know: "How much does Weng earn per minute?".\n\n',
 'Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?\nA: To answer the question "How much more money does Betty need to buy the wallet?", we need to know: "How much money does Betty have in the beginning?", "How much money did Betty\'s

In [54]:
revised["solving_pool"] = solving_pool
revised["decomposition_pool"] = decomposition_pool
with open("examples/rap_gsm8k_l2m/prompts/l2m_standard.json", "w") as f:
    json.dump(revised, f, indent=4)